In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.data import Dataset
#physical_devices = tf.config.list_physical_devices('GPU')
#tf0.config.experimental.set_memory_growth(physical_devices[0], True)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
%aimport denoiser.data
from denoiser import data

In [4]:
path = 'D:\\Amit\\Documents\\GitHub\\denoiser\\egs\\debug\\tr\\'
files_ds = data.load_files(path)
list(files_ds.as_numpy_iterator())

[(b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\noisy\\p287_001.wav',
  b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\clean\\p287_001.wav'),
 (b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\noisy\\p287_002.wav',
  b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\clean\\p287_002.wav'),
 (b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\noisy\\p287_003.wav',
  b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\clean\\p287_003.wav'),
 (b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\noisy\\p287_004.wav',
  b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\clean\\p287_004.wav'),
 (b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\noisy\\p287_005.wav',
  b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\clean\\p287_005.wav'),
 (b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\noisy\\p287_006.wav',
  b'D:\\Amit\\Documents\\GitHub\\denoiser\\dataset\\debug\\clean\\p287_006.wav')]

In [5]:
audio_ds = data.generate_audio(files_ds)
audio = list(audio_ds.as_numpy_iterator())
[(a.shape,b.shape) for a,b in audio]

NotImplementedError: in user code:

    D:\Amit\Documents\GitHub\denoiser\denoiser\data.py:89 apply_all  *
        res = tuple(func(*source) if isinstance(source, tuple) else func(source) for source in sources)
    D:\Amit\Documents\GitHub\denoiser\denoiser\data.py:97 read_audio  *
        audio = tfio.audio.AudioIOTensor(file_path,dtype='int16').to_tensor()
    C:\Users\Amit\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow_io\core\python\ops\audio_ops.py:236 __init__  **
        resource = core_ops.io_audio_readable_init(filename)
    C:\Users\Amit\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow_io\core\python\ops\__init__.py:87 __getattr__
        return getattr(self._load(), attrb)
    C:\Users\Amit\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow_io\core\python\ops\__init__.py:83 _load
        self._mod = _load_library(self._library)
    C:\Users\Amit\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow_io\cor

In [6]:
%aimport denoiser.demucs
from denoiser.demucs import Demucs

In [7]:
segment = 2
stride = 0.5

model = Demucs(hidden=16)

length = model.valid_length(segment*16_000)
stride = int(stride * 16_000)
print(length, stride)

32085 8000


In [8]:
examples_ds = data.generate_examples(audio_ds, length=length, stride=stride, batch=6)
[(a.shape,b.shape) for a,b in examples_ds.as_numpy_iterator()]

NameError: name 'audio_ds' is not defined

In [9]:
model.compile(optimizer='adam',loss='mse')
model.build((None,None,1))
model.summary()

[<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x00000190C7DBFA30>, <tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x00000190C7DC8D30>]
Model: "demucs"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
upsample4 (Sequential)       (None, None, None)        0         
_________________________________________________________________
downsample4 (Sequential)     (None, None, 1)           0         
_________________________________________________________________
normalize (Normalize)        multiple                  0         
_________________________________________________________________
Encode_1 (Sequential)        (None, None, 16)          688       
_________________________________________________________________
Encode_2 (Sequential)        (None, None, 32)          6240      
_________________________________________________________________
Encode_3 (Sequential)        

In [ ]:
model.fit(examples_ds, epochs=3)